***stock Evaluation***
********************************************************************************
**This script aims to evaluate, based on the hystorical date of an stock, how much is convenient to reduce the occurence of the investment in the stock**

***Libraries Definition***

In [4]:
from modules import general_utils
from modules import stock_utils
import pandas as pd
from pandas import DataFrame
import warnings

***MAIN CODE***
===========================

Hystorical Behavior of the Selected stock
===========================

In [11]:

### GENERIC INFORMATION ABOUT stock and its HYSTORICAL BEHAVIOR
# Time Informations
day, month, year = stock_utils.month_year()
my_portfolio_analysis = int(
    input("Enter 0 for Single Stock Analysis or 1 for My Portfolio Analysis")
)
if my_portfolio_analysis:
    github_raw_url = "https://raw.githubusercontent.com/giuseppedavidde/Data_for_Analysis/main/My_Portfolio.csv"
    stock_under_test = stock_utils.collect_data_from_github(github_raw_url)
else:
    stock_under_test = input("Enter the stock ticker symbol: ")
    category = input("Insert Category ETF, Crypto, Stock")
    start_date = input("Enter the start date (YYYY-MM-DD): ")
end_date = f"{year}-{month}-{day}"
if isinstance(stock_under_test, pd.DataFrame):
    stock_data_object = {}
    for symbol in stock_under_test["Symbol"]:
        if pd.notna(symbol):
            stock_data_object[symbol] = stock_utils.get_stock_data(symbol)
else:
    stock_data_object = stock_utils.get_stock_data(stock_under_test)
    print(stock_data_object.history(period="1d"))

                            Open   High    Low  Close  Volume  Dividends  \
Date                                                                       
2025-02-28 00:00:00+01:00 6.6850 6.7100 6.6500 6.6810   84507     0.0000   

                           Stock Splits  Capital Gains  
Date                                                    
2025-02-28 00:00:00+01:00        0.0000         0.0000  


Evaluation with DCA with Custom Purchase Frequency
===========================

User Input Data
===============

In [7]:
from datetime import datetime as dtm
pd.set_option('display.float_format', '{:.4f}'.format)
### Percentuale di peso sul portafoglio
### Ribilanciare in automatico (in una altra sezione!)
if isinstance(stock_under_test, pd.DataFrame):
    stock_data_dca_values = {}
    end_date_dca = f"{year}-{month}-{day}"
    ### Capitale Investito per ogni ETF/Stock/Crypto
    invested_capital_usd_dict = {}
    invested_capital_eur_dict = {}
    invested_capital_eur_dict_today = {}
    shares_dict = {}
    shares_cost_dict = {}
    shares_value_dict = {}
    gain_shares_absolute_value_dict = {}
    gain_shares_percentage_dict = {}
    capital_by_category = {category: 0.0 for category in stock_under_test["Category"].unique()}
    capital_by_category_today = {category: 0.0 for category in stock_under_test["Category"].unique()}
    weight_by_symbol = {}
    weight_by_symbol_today = {}
    total_gain = 0.0
    #############################
    # USDEUR DailyRate Change
    forex_data = stock_data_object["USDEUR=X"].history(period="1d")
    latest_rate = forex_data["Close"].iloc[-1]
    #############################    
    for symbol in stock_under_test["Symbol"]:
        if symbol != "USDEUR=X":
            category = stock_under_test.loc[stock_under_test["Symbol"] == symbol, "Category"].iloc[0]
            shares = stock_under_test.loc[stock_under_test["Symbol"] == symbol, "Shares"].iloc[0]
            invested_capital_usd = stock_under_test.loc[stock_under_test["Symbol"] == symbol, "Invested (USD)"].iloc[0]
            invested_capital_eur = stock_under_test.loc[stock_under_test["Symbol"] == symbol, "Invested (EUR)"].iloc[0]
            invested_capital_usd_dict[symbol] = invested_capital_usd
            shares_dict[symbol] = shares
            shares_info = stock_data_object[symbol].history(period="1d")
            if invested_capital_usd_dict[symbol] > 0.0:
                shares_value_dict[symbol] = shares_info["Close"].iloc[-1] * latest_rate 
            elif symbol == "IE00BK5BQT80" or symbol == "IE00BFMXXD54" :
                shares_value_dict[symbol] = shares_info["Close"].iloc[-1] * latest_rate 
            else:
                shares_value_dict[symbol] = shares_info["Close"].iloc[-1]
            invested_capital_eur_dict[symbol] = (invested_capital_usd_dict[symbol] * latest_rate + invested_capital_eur ) if invested_capital_usd_dict[symbol] > 0.0 else (invested_capital_eur)
            invested_capital_eur_dict_today[symbol] = (shares * shares_value_dict[symbol] ) 
            shares_cost_dict[symbol] = invested_capital_eur_dict[symbol]/shares_dict[symbol] if shares_dict[symbol] > 0.0 else 0.0
            gain_shares_absolute_value_dict[symbol] = (shares_value_dict[symbol]*shares_dict[symbol]) - invested_capital_eur_dict[symbol]
            gain_shares_percentage_dict[symbol] = (gain_shares_absolute_value_dict[symbol]/invested_capital_eur_dict[symbol])*100
            total_gain += gain_shares_absolute_value_dict[symbol]
            if category not in capital_by_category:
                capital_by_category[category] = 0.0
            capital_by_category[category] += invested_capital_eur_dict[symbol]
            capital_by_category_today[category] += invested_capital_eur_dict_today[symbol]
            weight_by_symbol[symbol] = {"invested": invested_capital_eur_dict[symbol], "category": category}
            weight_by_symbol_today[symbol] = {"invested": invested_capital_eur_dict_today[symbol], "category": category}
            ####### STOCK DATA based on Last Purchase Date ######    
            stock_data_dca_values[symbol] = stock_utils.get_stock_with_date_index_data(stock_data_object[symbol], category, stock_under_test.loc[stock_under_test["Symbol"] == symbol,"Last Purchase (YY-MM-DD)"].iloc[0], end_date_dca, ma_period=200)
            #####################################################
    invested_capital_usd = stock_under_test["Total Invested (USD)"].iloc[0]
    invested_capital_eur = stock_under_test["Total Invested (EUR)"].iloc[0]
    invested_capital_eur += (invested_capital_usd*latest_rate) 
    #Bisogna avere invested_capital diviso per ogni stock/ETF per poter procedere con il get_info_investment
    df_invested_capital = pd.DataFrame({
    'Symbol': list(invested_capital_eur_dict.keys()),
    'BEGINNING: Invested Capital (EUR)': list(invested_capital_eur_dict.values()),
    'TODAY: Invested Capital (EUR)': list(invested_capital_eur_dict_today.values()),
    'Shares': list(shares_dict.values()),
    'Share Value (EUR)': list(shares_value_dict.values()),
    'Share Cost (EUR)': list(shares_cost_dict.values()),
    'Gain (EUR)': list(gain_shares_absolute_value_dict.values()),
    'Gain %': list(gain_shares_percentage_dict.values()),
    })
    # Stampa il DataFrame
    #show_table(df_invested_capital)
    print(df_invested_capital.to_string(index=False))
    invested_capital_invested = float(invested_capital_eur)
    print(f"\n invested_capital EUR : {invested_capital_invested}")
    print(f"\n Total Return (EUR): {float(total_gain)}")
    print(f"\n Total Return %: {float(total_gain/invested_capital_invested)*100}")
    
    # Calcolo delle percentuali di peso per simbolo
    for symbol, data in weight_by_symbol.items():
        category_total = capital_by_category[data["category"]]
        if category_total > 0.0:
            weight_by_symbol[symbol]["weight"] = data["invested"] / category_total * 100
        else:
            weight_by_symbol[symbol]["weight"] = 0

    # Calcolo delle percentuali di peso per simbolo
    for symbol, data in weight_by_symbol_today.items():
        category_total_today = capital_by_category_today[data["category"]]
        if category_total_today > 0.0:
            weight_by_symbol_today[symbol]["weight"] = data["invested"] / category_total_today * 100
        else:
            weight_by_symbol_today[symbol]["weight"] = 0
    
    # Utilizzo della funzione generica per il capitale totale investito per categoria
    categories = list(capital_by_category.keys())
    category_values = list(capital_by_category.values())
    category_values_today = list(capital_by_category_today.values())
    stock_utils.create_pie_chart(capital_by_category, categories, category_values, 'BEGINNING: Capitale Totale Investito per Categoria')
    stock_utils.create_pie_chart(capital_by_category_today, categories, category_values_today, 'TODAY: Capitale Totale Investito per Categoria')

    # Utilizzo della funzione generica per il peso percentuale di ogni simbolo all'interno della sua categoria
    for category in capital_by_category.keys():
        if category == "Forex":  # Salta la categoria 'Forex'
            continue
        category_symbols = [symbol for symbol, data in weight_by_symbol.items() if data['category'] == category]
        category_weights = [data['weight'] for symbol, data in weight_by_symbol.items() if data['category'] == category]
        category_weights_today = [data['weight'] for symbol, data in weight_by_symbol_today.items() if data['category'] == category]
        if category_symbols and category_weights:  # Assicurati che ci siano dati da plottare
            stock_utils.create_pie_chart(weight_by_symbol, category_symbols, category_weights, f'BEGINNING: Peso Percentuale per Simbolo in Categoria "{category}"')
        if category_symbols and category_weights_today:  # Assicurati che ci siano dati da plottare
            stock_utils.create_pie_chart(weight_by_symbol_today, category_symbols, category_weights_today, f'TODAY: Peso Percentuale per Simbolo in Categoria "{category}"')

    ## In base alla categoria, posso identificare quanto voglio investire e calcolare il nuovo DCA, basandomi sul costo giornaliero, cosi da poter trovare la variazione piu grande di costo di carico e ottimizzare gli acquisti
    
    
else:
        invested_capital_invested = float(input("Insert how much you want to invest in EUR"))
        start_date_dca = input("Enter the start date for DCA Strategy (YYYY-MM-DD): ")
        ##end_date_dca = input("Enter the end date for DCA Strategy (YYYY-MM-DD): ")
        end_date_dca = f"{year}-{month}-{day}"
        stock_data_dca_values = stock_utils.get_stock_with_date_index_data(
        stock_data=stock_data_object, category= category, start_date=start_date_dca, end_date=end_date_dca, ma_period=200
        )

Plot Stock/ETF with some statistics
=====================================

In [7]:
plot_stock_under_test = int(
    input("Enter 0 to skip Plot or 1 to Plot your Portfolio ETFs/Stocks")
)
if plot_stock_under_test:
    if isinstance(stock_data_dca_values, dict):
        for symbol, df in stock_data_dca_values.items():
            dates = df.index  # Prendi l'indice del DataFrame per il simbolo corrente
            
            # Prepara i dati per il plot
            plot_data = {
                "y": [df["stock_price"], df["MA200"], df["OBV"]],
                "x": [dates, dates, dates],  # Stesse date per tutte le serie
                "name_trace": [f"{symbol} Stock Price", f"{symbol} MA200", f"{symbol} OBV"],
                "name_graph": f"{symbol} Stock Price and Moving Average and OBV",
                "xaxis_title": "Date",
                "yaxis_title": "Price",
            }

            # Chiamata alla funzione 'create_plot'
            fig = stock_utils.create_plot(
                x=plot_data["x"],
                y=plot_data["y"],
                name_graph=plot_data["name_graph"],
                name_trace=plot_data["name_trace"],
                xaxis_title=plot_data["xaxis_title"],
                yaxis_title=plot_data["yaxis_title"],
                )
    else: 
        dates = stock_data_dca_values.index
        plot_data = {
            "y": [
                stock_data_dca_values["stock_price"],
                stock_data_dca_values["MA200"],
                stock_data_dca_values["OBV"],
            ],
            "x": [dates, dates, dates],  # Usa le stesse date per stock_price e MA200
            "name_trace": ["Stock Price", "MA200", "OBV"],  # Nomi delle serie
            "name_graph": f"{stock_under_test} Stock Price and Moving Average and OBV",
            "xaxis_title": "Date",
            "yaxis_title": "Price in USD",
        }
        # Chiamata alla funzione 'create_plot'
        fig = stock_utils.create_plot(
            x=plot_data["x"],
            y=plot_data["y"],
            name_graph=plot_data["name_graph"],
            name_trace=plot_data["name_trace"],
            xaxis_title=plot_data["xaxis_title"],
            yaxis_title=plot_data["yaxis_title"],
        )



Calculations
========================

In [8]:
# Supponendo che start_date_dca e end_date_dca siano gia' stati inseriti e siano nel formato corretto.

from modules import general_utils
import pprint

from numpy import dtype ##To print Dictionary in Python

end_date = dtm.strptime(end_date_dca, "%Y-%m-%d")
if isinstance(stock_data_dca_values, dict):
    delta_months = {}
    start_date = {}
    for symbol in stock_under_test["Symbol"]:
        start_date[symbol] = dtm.strptime(stock_under_test.loc[stock_under_test["Symbol"]==symbol,"Last Purchase (YY-MM-DD)"].iloc[0], "%Y-%m-%d")
        delta_months[symbol] = ((end_date.year - start_date[symbol].year) * 12 + end_date.month - start_date[symbol].month)
        # Se vuoi considerare anche i giorni per un conteggio più preciso (se il giorno di fine è dopo il giorno di inizio)
        if end_date.day > start_date[symbol].day: delta_months[symbol] += 1
        # print(f"The number of months for {symbol} between start_date {stock_under_test.loc[stock_under_test["Symbol"]==symbol,"Last Purchase (YY-MM-DD)"].iloc[0]} and end_date {end_date_dca} is {delta_months}.")
        purchase_frequencies = [
        f"{delta_months[symbol]}M",  ##SUM LUMP
        ]  # range of purchase frequencies   
else:
    start_date = dtm.strptime(start_date_dca, "%Y-%m-%d")
    # Calcolo della differenza in mesi per trovare il valore di mesi per effettuare un solo acquisto (LAMP SUM approach)
    delta_months = ((end_date.year - start_date.year) * 12 + end_date.month - start_date.month)    
    print(f"The number of months between {start_date_dca} and {end_date_dca} is {delta_months}.")
    stock_data_object = {symbol: stock_utils.get_stock_data(symbol) for symbol in stock_under_test["Symbol"] if pd.notna(symbol)}
    purchase_frequencies = [
    "1M",
    "3M",
    "6M",
    "9M",
    "12M",
    "15M",
    f"{delta_months}M",  ##SUM LUMP
    ]  # range of purchase frequencies
global_market_values                 = {freq: pd.Series(dtype=float) for freq in purchase_frequencies}
total_market_value_by_category       = {freq: pd.Series(dtype=float) for freq in purchase_frequencies}
category_dict                        = {category: pd.Series(dtype=str) for category in stock_under_test["Category"].unique()}
results                              = {freq: pd.Series(dtype=float) for freq in purchase_frequencies}
weight_by_category_today             = {freq: pd.Series(dtype=float) for freq in purchase_frequencies}
weight_by_symbol_today               = {freq: pd.Series(dtype=float) for freq in purchase_frequencies}

for freq in purchase_frequencies:
    if isinstance(stock_data_dca_values, dict):
        for symbol in stock_under_test["Symbol"]:
            if symbol != "USDEUR=X":
                usd_investment = 0  # Resetta la flag per ogni nuovo symbol
                start_date_symbol = stock_under_test.loc[stock_under_test["Symbol"] == symbol, "Last Purchase (YY-MM-DD)"].iloc[0]
                if invested_capital_usd_dict[symbol] > 0.0 or symbol == "IE00BK5BQT80" or symbol == "IE00BFMXXD54":
                    result = stock_utils.get_info_investment(

                        stock_data_dca_values[symbol],
                        invested_capital_usd_dict[symbol],
                        start_date_symbol,
                        end_date_dca,
                        freq,
                    )
                    usd_investment = 1
                elif invested_capital_eur_dict[symbol] > 0.0:
                    result = stock_utils.get_info_investment(
                        stock_data_dca_values[symbol],
                        invested_capital_eur_dict[symbol],
                        start_date_symbol,
                        end_date_dca,
                        freq,
                    )
                results[freq][symbol] = result
                if usd_investment == 1 and "market_value" in result:
                    results[freq][symbol]["market_value"] *= latest_rate

        print(f"Results : {results}")
        
        # Aggiorna global_market_values dopo aver elaborato tutti i simboli per la frequenza corrente   
        # for symbol, result in results[freq].items():
        #     if "market_value" in result:
        #         if global_market_values[freq].empty:
        #             global_market_values[freq] = result["market_value"]
        #         else:
        #             global_market_values[freq] = global_market_values[freq].add(result["market_value"], fill_value=0)
           
        # for category_today in category_dict.keys():
        #     if category_today == "Forex":  # Salta la categoria 'Forex'
        #         continue
        #     total_market_value_by_category[freq][category_today] = sum(
        #         result.get("market_value", 0).iloc[-1]
        #         for symbol, result in results[freq].items()
        #             if weight_by_symbol[symbol]["category"] == category_today
        #         )

        # for category_today in category_dict.keys():
        #     # Calcolo del peso percentuale di ogni simbolo rispetto al totale di market_value per ciascuna categoria e frequenza
        #     for symbol in results[freq].keys(): 
        #         if weight_by_symbol[symbol]["category"] == category_today:
        #             symbol_market_value = results[freq][symbol].get("market_value", 0).iloc[-1]
        #             category_total_market_value = total_market_value_by_category[freq][category_today]
        #             if category_total_market_value > 0.0 :  # Assicurati che sia un valore scalare
        #                 weight_by_symbol_today[freq][symbol]["weight"] = (symbol_market_value / category_total_market_value) * 100
        #                 weight_by_symbol_today[freq][symbol]["category"] = category_today

    # Utilizzo della funzione generica per il peso percentuale di ogni simbolo all'interno della sua categoria
        # for category in capital_by_category.keys():
        #     if category == "Forex":  # Salta la categoria 'Forex'
        #         continue
        #     category_symbols_today = [symbol for symbol, data in weight_by_symbol_today.items() if data['category'] == category]
        #     category_weights_today = [data['weight'] for symbol, data in weight_by_symbol_today.items() if data['category'] == category]
        #     if category_symbols_today and category_weights_today:  # Assicurati che ci siano dati da plottare
        #         stock_utils.create_pie_chart(weight_by_symbol_today, category_symbols_today, category_weights_today, f'Peso Percentuale per Simbolo in Categoria "{category}"')
#
    #
        
    else:
        results = {}
        global_market_values = {freq: pd.Series(dtype=float) for freq in purchase_frequencies}
        # Calcolo del totale di market_value per categoria e frequenza
        weight_by_symbol_today      = {symbol: {} for symbol in stock_under_test["Symbol"]}
        weight_by_category_today    = {category: stock_under_test.loc[stock_under_test["Symbol"] == symbol, "Category"].iloc[0] for symbol in stock_under_test["Symbol"]}
        total_market_value_by_category= {}

        result = stock_utils.get_info_investment(
            stock_data_dca_values,
            invested_capital_invested,
            start_date_dca,
            end_date_dca,
            freq,
        )
        results[freq] = result




## pp = pprint.PrettyPrinter(indent=4)
## pp.pprint(results)
    #
# for freq, market_value_series in global_market_values.items():
#     print(f"Frequency: {freq}")
#     print(market_value_series)
#print(f"{results["1ME"]["GME"]}")

###### TROVARE IL VALORE DI INVESTIMENTO DA EFFETTUARE PER RIPORTARE A 0 LA MINUSVALENZA!
## num_azioni_attuali = 402.68
## costo_medio_attuale = 7.93
## prezzo_attuale = 1.02
## 
## investimento_aggiuntivo = (num_azioni_attuali * (costo_medio_attuale - prezzo_attuale)) / (1 - (1 / prezzo_attuale))
## 
## print(f"L'investimento aggiuntivo necessario è: {investimento_aggiuntivo:.2f} USD")


The number of months between 2025-02-28 and 2025-3-2 is 1.


TypeError: string indices must be integers, not 'str'

Plotting Section for DCA vs Sum Lump 
====================================

In [9]:
from dataclasses import field


plot_stock_under_test = int(
    input("Enter 0 to skip Plot or 1 to Plot your DCA vs Sum Lump ETFs/Stocks")
)
if plot_stock_under_test:
       
     # Creare i widget per l'interattività
    if isinstance(stock_data_dca_values, dict):
        fields_to_plot = ["average_cost", "market_value", "daily_gain", "daily_gain_perc","global_market_values"]
    else:
        fields_to_plot = ["average_cost", "market_value", "daily_gain", "daily_gain_perc"]
        
    field_selector = general_utils.widgets.Dropdown(options=fields_to_plot, value='market_value', description='Field')
    freq_selector = general_utils.widgets.Dropdown(options=purchase_frequencies, value=purchase_frequencies[-1], description='Frequency')
    # Usa 'interactive' per collegare i widget alla funzione di creazione dei grafici
    interactive_plot = general_utils.interactive(stock_utils.create_interactive_plot,
                                   stock_data_df_or_dict=general_utils.fixed(stock_data_dca_values),
                                   results=general_utils.fixed(results),
                                   field=field_selector,
                                   exception_field_string=general_utils.fixed("global_market_values"),
                                   exception_field_array=general_utils.fixed(global_market_values),
                                   freq=freq_selector,
                                   )

    display(interactive_plot) 

     ##Define which is the best strategy
    # (best_strategy, best_average_cost, best_number_shares, best_market_value) = (
    #     get_best_investment_strategy(results)
    # )
